In [7]:

from matplotlib import pyplot as plt
import pandas as pd
from utils import save_fig_for_raport, dataframe_to_latex_tabular 
from pandas.plotting import autocorrelation_plot
import pandas.plotting
from pandas.plotting import lag_plot
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import StandardScaler

In [2]:

raw_data = pd.read_csv('../datasets/raw/market_insights_overview_all_2022_04_26_filtered.csv', parse_dates=["date"])

# Filter away early 2018 data as it seems to contain errors
raw_data =raw_data .loc[raw_data ['date'] > '2018-12-01']

In [3]:

categories = pd.read_csv("../datasets/raw/solr_categories_all_2022_02_14.csv")
# Join market_insight table with category names
category_names = categories[[ "internal_doc_id", "title"]]
category_id_to_names_map = dict(map(lambda x: (x["internal_doc_id"], x["title"]), category_names.to_dict("records")))
category_name_to_id_map = dict(map(lambda x: (x["title"], x["internal_doc_id"]), category_names.to_dict("records")))

In [4]:


data_grouped = raw_data.groupby(['cat_id', "date"], as_index=False).sum()
data_grouped["date"] = pd.to_datetime(data_grouped["date"])
data_grouped["interest"] = data_grouped["hits"] + data_grouped["clicks"]

# Join with category names
data_grouped = pd.merge(data_grouped , category_names, how="left", left_on=["cat_id"], right_on=["internal_doc_id"])
data_grouped.rename(columns={"title": "cat_name"}, inplace=True)
data_grouped.drop(columns=["internal_doc_id"], inplace=True, axis=1)

data_chosen_categories = data_grouped[["interest", "date", "cat_id", "cat_name"]]
data = data_chosen_categories

In [37]:
import numpy as np
minimize_outliers = lambda x: x if (x <= (4 *std)) else  x * 0.3
vector_func = np.vectorize(minimize_outliers)

max_variance_threshold = 0.8
min_variance_threshold = 0.3
larg_variance_ids = []
small_variance_ids = []
for cat_id in data["cat_id"].unique():
  filtered = data[data["cat_id"] == cat_id]
  std = filtered["interest"].std()
  filtered["interest"] = filtered["interest"].apply(minimize_outliers)

  scaler = StandardScaler()
  scaled = scaler.fit_transform(filtered["interest"].values.reshape(-1, 1))
  diff = pd.DataFrame(scaled).diff()
  std = diff.std()[0]
  if std > max_variance_threshold:
    print(f"Category {cat_id} is too noisy with std {std}")
    larg_variance_ids.append(cat_id)
    # diff.plot()
    # print("std", std)
    # plt.show()
  if std < min_variance_threshold:
    print(f"Category {cat_id} is too quiet with std {std}")
    small_variance_ids.append(cat_id)
    diff.plot()
    print("std", std)
    plt.show()



/tmp/ipykernel_14748/122176398.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered["interest"] = filtered["interest"].apply(minimize_outliers)
/tmp/ipykernel_14748/122176398.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered["interest"] = filtered["interest"].apply(minimize_outliers)
/tmp/ipykernel_14748/122176398.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

Category 2 is too noisy with std 0.8517802130027626
Category 6 is too noisy with std 0.9542587048854919
Category 9 is too noisy with std 0.825114673577595
Category 10 is too noisy with std 0.8922583069759911
Category 11 is too noisy with std 1.0579081870855649
Category 13 is too noisy with std 1.0072842275551273
Category 20 is too noisy with std 0.8778373375620367
Category 24 is too noisy with std 0.9034376600720286
Category 26 is too noisy with std 0.9171783313849785
Category 27 is too noisy with std 0.823688969409603
Category 28 is too noisy with std 1.0890789434619523
Category 33 is too noisy with std 0.8420961455280503
Category 34 is too noisy with std 0.9187120569134944
Category 41 is too noisy with std 0.8686468126881352
Category 46 is too noisy with std 0.8682380627757783
Category 54 is too noisy with std 1.079559495750459
Category 11694 is too noisy with std 0.8269762662207288
Category 11950 is too noisy with std 1.1528202122457283
Category 11998 is too noisy with std 1.0959907

/tmp/ipykernel_14748/122176398.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered["interest"] = filtered["interest"].apply(minimize_outliers)
/tmp/ipykernel_14748/122176398.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered["interest"] = filtered["interest"].apply(minimize_outliers)
/tmp/ipykernel_14748/122176398.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 